# 03 · Retrieval Visualization（Legal-RAG）

本 Notebook 用于直观查看 Legal-RAG 在《民法典·合同编》上的检索效果：

- 对同一个法律问题，对比：
  - **BM25 only**（纯文本检索）
  - **Dense-only**（BGE + FAISS）
  - **Hybrid**（dense + BM25 融合）
- 用图表展示 Hybrid 检索得分分布
- 额外：展示某个命中条文在 `law_graph` 中的邻居条文（如有）

> 说明：本 Notebook **不调用 LLM**，只关注“检索质量的可视化”。

## 1. 环境与配置

- 需要提前已经运行过：
  - `python -m scripts.preprocess_law`
  - `python -m scripts.build_index`
- 已经构建：
  - `data/processed/contract_law.jsonl`
  - `data/index/faiss.index` + `faiss_meta.jsonl`
  - `data/index/bm25.pkl`


In [ ]:
from pathlib import Path
from typing import List, Dict, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from legalrag.config import AppConfig
from legalrag.models import RetrievalHit, LawChunk
from legalrag.retrieval.bm25_retriever import BM25Retriever
from legalrag.retrieval.vector_store import VectorStore
from legalrag.retrieval.hybrid_retriever import HybridRetriever
from legalrag.retrieval.graph_store import LawGraphStore
from legalrag.utils.logger import get_logger

%matplotlib inline

logger = get_logger(__name__)

# 加载配置
cfg = AppConfig.load()
BASE_DIR = Path(cfg.paths.base_dir)
DATA_DIR = Path(cfg.paths.data_dir)
INDEX_DIR = Path(cfg.paths.index_dir)

print("BASE_DIR:", BASE_DIR)
print("DATA_DIR:", DATA_DIR)
print("INDEX_DIR:", INDEX_DIR)


## 2. 初始化检索组件

这里初始化三种检索器，以及可选的 `LawGraphStore`：

- `BM25Retriever` – 关键词 / 倒排索引
- `VectorStore` – BGE 向量 + FAISS
- `HybridRetriever` – 两者加权融合
- `LawGraphStore` – 用于观察法条图中邻居条文（如果 graph JSONL 已构建）

In [ ]:
bm25 = BM25Retriever(cfg)
vector_store = VectorStore(cfg)
hybrid = HybridRetriever(cfg)

try:
    graph_store = LawGraphStore(cfg)
    HAS_GRAPH = True
    print("LawGraphStore loaded.")
except Exception as e:
    graph_store = None
    HAS_GRAPH = False
    print("LawGraphStore not available:", e)

print("BM25, VectorStore, Hybrid initialized.")

## 3. 单问题多检索器对比

- BM25
- Dense-only（VectorStore）
- Hybrid

结果整理成 pandas DataFrame，方便可视化。

In [ ]:
def run_all_retrievers(question: str, top_k: int = 5) -> Dict[str, List[RetrievalHit]]:
    """对同一个 question 调用三种检索器，返回 hit 列表。"""

    bm25_hits: List[RetrievalHit] = []
    for idx, (chunk, score) in enumerate(bm25.search(question, top_k=top_k), start=1):
        bm25_hits.append(
            RetrievalHit(
                chunk=chunk,
                score=float(score),
                rank=idx,
                source="bm25",
            )
        )

    dense_hits: List[RetrievalHit] = []
    for idx, (chunk, score) in enumerate(vector_store.search(question, top_k=top_k), start=1):
        dense_hits.append(
            RetrievalHit(
                chunk=chunk,
                score=float(score),
                rank=idx,
                source="dense",
            )
        )

    hybrid_hits: List[RetrievalHit] = hybrid.search(question, top_k=top_k)
    for h in hybrid_hits:
        h.source = "hybrid"

    return {
        "bm25": bm25_hits,
        "dense": dense_hits,
        "hybrid": hybrid_hits,
    }


def hits_to_dataframe(hits: List[RetrievalHit], retriever_name: str) -> pd.DataFrame:
    rows = []
    for h in hits:
        c = h.chunk
        rows.append(
            {
                "retriever": retriever_name,
                "rank": h.rank,
                "score": float(h.score),
                "article_no": getattr(c, "article_no", ""),
                "chapter": getattr(c, "chapter", ""),
                "section": getattr(c, "section", ""),
                "preview": (c.text or "").replace("\n", " ")[:80] + "...",
            }
        )
    return pd.DataFrame(rows)


## 4. 定义一组典型示例问题

这里准备几条典型的合同法问题：

1. 违约金比例是否过高
2. 不可抗力
3. 解除合同
4. 定金责任
5. 合同无效情形

后面会对这些问题做多轮对比。

In [ ]:
sample_questions = [
    "合同约定的违约金为合同金额的 40%，是否合理？",
    "什么是不可抗力？",
    "合同一方迟延履行，另一方在什么条件下可以解除合同？",
    "当事人约定了定金，违约时定金如何处理？",
    "哪些情形下合同会被认定为无效？",
]

for idx, q in enumerate(sample_questions, start=1):
    print(f"[{idx}] {q}")

## 5. 对单个问题进行详细对比

选择一个问题，查看三种检索器的 top-k 结果：

- 每条结果的 rank / score
- 条号 / 章节
- 文本预览（前 80 字）


In [ ]:
q_idx = 1  # 修改这个索引选择不同问题（1 ~ len(sample_questions)）
top_k = 5

question = sample_questions[q_idx - 1]
print("问题:", question)
print("=" * 80)

results = run_all_retrievers(question, top_k=top_k)

df_bm25 = hits_to_dataframe(results["bm25"], "bm25")
df_dense = hits_to_dataframe(results["dense"], "dense")
df_hybrid = hits_to_dataframe(results["hybrid"], "hybrid")

print("[BM25] top-", top_k)
display(df_bm25)
print("\n[Dense-BGE] top-", top_k)
display(df_dense)
print("\n[Hybrid] top-", top_k)
display(df_hybrid)


## 6. 对 Hybrid 检索结果做得分可视化

这里专门对 Hybrid 的 top-k 结果画一个简单的柱状图：

- x 轴：rank
- y 轴：score
- 条文号作为 x 轴刻度标签

可以直观看到：
- Hybrid 的打分分布
- 哪些条文被排在前面，以及分差大概有多大。

In [ ]:
def plot_hybrid_scores(df_hybrid: pd.DataFrame, question: str):
    if df_hybrid.empty:
        print("No hybrid hits to plot.")
        return

    ranks = df_hybrid["rank"].values
    scores = df_hybrid["score"].values
    labels = df_hybrid["article_no"].values

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.bar(ranks, scores)
    ax.set_xticks(ranks)
    ax.set_xticklabels(labels, rotation=30, ha="right")
    ax.set_xlabel("Article No (ranked)")
    ax.set_ylabel("Hybrid score")
    ax.set_title("Hybrid Retrieval Scores\n" + question)
    ax.grid(axis="y", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.show()


plot_hybrid_scores(df_hybrid, question)

## 7. law_graph 邻居条文可视化

需要先通过 `scripts/build_graph.py` 构建：

- `data/graph/law_graph.jsonl`

这里：

1. 取 Hybrid 第 1 个命中条文作为种子；
2. 在图中找出它的一层 / 两层邻居；
3. 打印邻居条文的条号 / 章节 / 预览内容；

这可以用来解释：

- Graph-aware 检索为何会把某些条文一起拉进上下文；
- 哪些条文属于“结构上相关（同章 / 同节 / 引用关系）”。

> 如果 `LawGraphStore` 未成功加载，本节会自动跳过。

In [ ]:
if not HAS_GRAPH:
    print("LawGraphStore not available, skip graph visualization.")
else:
    if df_hybrid.empty:
        print("No hybrid hits; please run the previous cell with a valid question.")
    else:
        # 取 Hybrid 第一条命中
        top_hit = results["hybrid"][0]
        seed_chunk = top_hit.chunk
        seed_id = getattr(seed_chunk, "id", None) or getattr(seed_chunk, "article_id", None)
        print("Seed article id:", seed_id)
        print("Seed article no:", getattr(seed_chunk, "article_no", ""))
        print("Seed chapter/section:", getattr(seed_chunk, "chapter", ""), getattr(seed_chunk, "section", ""))
        print("Seed preview:", (seed_chunk.text or "").replace("\n", " ")[:120] + "...")
        print("-" * 80)

        neighbors = []
        try:
            # 优先使用 get_neighbors(seed_id, depth=1)
            neighbors = graph_store.get_neighbors(seed_id, depth=1)
        except TypeError:
            # 兼容只接受 (id) 的旧接口
            neighbors = graph_store.get_neighbors(seed_id)
        except Exception as e:
            print("get_neighbors failed:", e)
            neighbors = []

        print(f"Graph neighbors (depth=1): {len(neighbors)} 条")
        rows = []
        for n in neighbors:
            rows.append(
                {
                    "article_id": getattr(n, "article_id", None) or getattr(n, "id", None),
                    "article_no": getattr(n, "article_no", ""),
                    "chapter": getattr(n, "chapter", ""),
                    "section": getattr(n, "section", ""),
                    "relations": getattr(n, "relations", None),
                    "preview": (getattr(n, "text", "") or "").replace("\n", " ")[:80] + "...",
                }
            )

        if rows:
            df_neighbors = pd.DataFrame(rows)
            display(df_neighbors)
        else:
            print("No graph neighbors found.")


## 8. 小结
- 实现 BM25 / BGE 向量 / Hybrid 三种检索管线，并制作可视化 Notebook 对比不同检索模式的命中条文和得分分布；
- 在示例合同法问题（如违约金比例、不可抗力、解除合同、定金责任）上展示 top-k 检索条文及章节信息，便于法律专家人工审查；
- 集成 `law_graph`，支持从命中条文出发查看图结构中的邻居条文，方便解释 Graph-aware RAG 为什么会扩展某些条文进入上下文。
